# Enumerating λ-join-calculus normal forms

In [ ]:
import math
from collections import Counter

import matplotlib.pyplot as plt

from hstar.enumeration import Refiner, enumerator
from hstar.normal import VAR, Term, complexity, is_deterministic, is_normal

%config InlineBackend.figure_format = 'svg'

This should take about 20 seconds on the first execution.

In [ ]:
%%time
max_complexity = 12
terms: list[Term] = []
for term in enumerator:
    if complexity(term) > max_complexity:
        break
    terms.append(term)
len(terms)

In [ ]:
closed_terms = [t for t in terms if not t.free_vars]
len(closed_terms)

In [ ]:
det_terms = [t for t in terms if is_deterministic(t)]
len(det_terms)

In [ ]:
closed_det_terms = [t for t in closed_terms if is_deterministic(t)]
len(closed_det_terms)

In [ ]:
normal_terms = [t for t in terms if is_normal(t)]
len(normal_terms)

In [ ]:
closed_normal_terms = [t for t in closed_terms if is_normal(t)]
len(closed_normal_terms)

In [ ]:
%%time
sketches: list[Term] = []
refiner = Refiner(VAR(0), on_fact=lambda term, valid: None)
while True:
    term = refiner.next_candidate()
    if complexity(term) > max_complexity:
        break
    sketches.append(term)
len(sketches)

In [ ]:
def plot_counts() -> None:
    all_counts = Counter(complexity(t) for t in terms)
    closed_counts = Counter(complexity(t) for t in closed_terms)
    det_counts = Counter(complexity(t) for t in det_terms)
    closed_det_counts = Counter(complexity(t) for t in closed_det_terms)
    normal_counts = Counter(complexity(t) for t in normal_terms)
    closed_normal_counts = Counter(complexity(t) for t in closed_normal_terms)
    sketch_counts = Counter(
        complexity(t) for t in sketches if complexity(t) <= max_complexity
    )
    x = range(1, max_complexity + 1)
    y1 = [all_counts[i] for i in x]
    y2 = [closed_counts[i] for i in x]
    y3 = [det_counts[i] for i in x]
    y4 = [closed_det_counts[i] for i in x]
    y5 = [all_counts[i] - normal_counts[i] for i in x]
    y6 = [closed_counts[i] - closed_normal_counts[i] for i in x]
    y7 = [sketch_counts[i] for i in x]
    plt.figure(figsize=(8,6))
    plt.plot(x, y1, label="all", linestyle="--", color="C0")
    plt.plot(x, y2, label="closed", linestyle="-", color="C0")
    plt.plot(x, y3, label="deterministic", linestyle="--", color="C1")
    plt.plot(x, y4, label="closed deterministic", linestyle="-", color="C1")
    plt.plot(x, y5, label="non-normal", linestyle="--", color="C2")
    plt.plot(x, y6, label="closed non-normal", linestyle="-", color="C2")
    plt.plot(x, y7, label="sketches", linestyle="-", color="C3")
    plt.text(x[-1], y1[-1], f" {y1[-1]}", va="center", ha="left")
    plt.text(x[-1], y2[-1], f" {y2[-1]}", va="center", ha="left")
    plt.text(x[-1], y3[-1], f" {y3[-1]}", va="center", ha="left")
    plt.text(x[-1], y4[-1], f" {y4[-1]}", va="center", ha="left")
    plt.text(x[-1], y5[-1], f" {y5[-1]}", va="center", ha="left")
    plt.text(x[-1], y6[-1], f" {y6[-1]}", va="center", ha="left")
    plt.text(x[-1], y7[-1], f" {y7[-1]}", va="center", ha="left")
    plt.legend(loc="best")
    plt.xlim(1, max_complexity + 1)
    plt.yscale("log")
    # Add a grid of horizontal lines showing powers of 2.
    for i in range(0, int(math.log2(y1[-1])) + 1): 
        plt.axhline(2**i, color="k", linestyle="-", linewidth=0.5, alpha=0.15)
    plt.xlabel("Complexity")
    plt.ylabel("Count")
    plt.title("How many affine normal forms are there?")
    plt.show()

plot_counts()
